In [77]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/college-scorecard/MERGED2007_PP.csv
/kaggle/input/college-scorecard/MERGED2003_PP.csv
/kaggle/input/college-scorecard/Scorecard.csv
/kaggle/input/college-scorecard/MERGED1997_PP.csv
/kaggle/input/college-scorecard/hashes.txt
/kaggle/input/college-scorecard/MERGED1998_PP.csv
/kaggle/input/college-scorecard/MERGED2000_PP.csv
/kaggle/input/college-scorecard/MERGED2013_PP.csv
/kaggle/input/college-scorecard/MERGED2010_PP.csv
/kaggle/input/college-scorecard/FullDataDocumentation.pdf
/kaggle/input/college-scorecard/MERGED1996_PP.csv
/kaggle/input/college-scorecard/MERGED2006_PP.csv
/kaggle/input/college-scorecard/MERGED2008_PP.csv
/kaggle/input/college-scorecard/data_dictionary.yaml
/kaggle/input/college-scorecard/MERGED2012_PP.csv
/kaggle/input/college-scorecard/MERGED2001_PP.csv
/kaggle/input/college-scorecard/MERGED2004_PP.csv
/kaggle/input/college-scorecard/database.sqlite
/kaggle/input/college-scorecard/MERGED2011_PP.csv
/kaggle/input/college-scorecard/CollegeScorecardData

# The goal
What are common features among colleges and universities that have closed in the last decade?
Using these features, is it possible to predict which schools are in danger of closing in the near future?
Are there steps that schools can take to mitigate the features which may lead to their closing?

In [78]:
# Load additional packages and modules
import matplotlib.pyplot as plt
import seaborn as sns
#% matplotlib inline

# Data collection
The data come from the US Department of Education College Scorecard, which is available on Kaggle and from the US Department of Education at https://collegescorecard.ed.gov/data/.

In [79]:
# Load the most recent scorecard into a DataFrame for examination
# List of fields I want to import, based on previous exploration and data dictionary
fields = ['Id','OPEID','INSTNM','CITY', 'ZIP','sch_deg', 'main', 'NUMBRANCH', 'PREDDEG', \
         'HIGHDEG', 'CONTROL','st_fips','region','LOCALE','CCUGPROF','RELAFFIL', 'ADM_RATE',\
         'ACTCMMID','SAT_AVG', 'UGDS', 'UG', 'CURROPER', 'NPT4_PUB', 'NPT4_PRIV', 'COSTT4_A', \
         'COSTT4_P','TUITFTE', 'INEXPFTE', 'AVGFACSAL', 'PFTFAC',\
         'C150_4', 'C150_L4','RET_FT4','RET_FTL4','RET_PT4','RET_PTL4','PCTFLOAN','UG25abv',\
         'COMP_ORIG_YR4_RT','WDRAW_ORIG_YR4_RT', 'ENRL_ORIG_YR4_RT', 'INC_PCT_LO', 'DEP_STAT_PCT_IND',\
         'IND_INC_PCT_LO', 'DEP_INC_PCT_LO', 'PAR_ED_PCT_1STGEN', 'DEP_INC_AVG', 'IND_INC_AVG', \
         'DEBT_MDN', 'GRAD_DEBT_MDN', 'WDRAW_DEBT_MDN', 'loan_ever', 'age_entry', 'female',\
         'married', 'dependent', 'veteran', 'faminc', 'md_faminc', 'median_hh_inc']

data = pd.read_csv("../input/college-scorecard/Scorecard.csv", usecols=fields)
data.shape

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,7,20,25,36,807,810,813,1408,1409,1410,1411,1412,1432,1433,1504,1505,1506,1605,1607,1610,1611,1612,1613,1615,1616,1627) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(124699, 60)

## Data organization
Create a file structure (and add work to GitHub repo)

In [80]:
#Create directories for various components of the project
#os.mkdir("data")
#os.mkdir("data/raw")
#os.mkdir("data/interim")
#os.mkdir("data/processed")
#os.mkdir("data/external")
#os.mkdir("figures")
#os.mkdir("models")

In [81]:
data.columns

Index(['Id', 'OPEID', 'INSTNM', 'CITY', 'ZIP', 'sch_deg', 'main', 'NUMBRANCH',
       'PREDDEG', 'HIGHDEG', 'CONTROL', 'st_fips', 'region', 'LOCALE',
       'CCUGPROF', 'RELAFFIL', 'ADM_RATE', 'ACTCMMID', 'SAT_AVG', 'UGDS', 'UG',
       'CURROPER', 'NPT4_PUB', 'NPT4_PRIV', 'COSTT4_A', 'COSTT4_P', 'TUITFTE',
       'INEXPFTE', 'AVGFACSAL', 'PFTFAC', 'C150_4', 'C150_L4', 'RET_FT4',
       'RET_FTL4', 'RET_PT4', 'RET_PTL4', 'PCTFLOAN', 'UG25abv',
       'COMP_ORIG_YR4_RT', 'WDRAW_ORIG_YR4_RT', 'ENRL_ORIG_YR4_RT',
       'INC_PCT_LO', 'DEP_STAT_PCT_IND', 'DEP_INC_PCT_LO', 'IND_INC_PCT_LO',
       'PAR_ED_PCT_1STGEN', 'DEP_INC_AVG', 'IND_INC_AVG', 'DEBT_MDN',
       'GRAD_DEBT_MDN', 'WDRAW_DEBT_MDN', 'loan_ever', 'age_entry', 'female',
       'married', 'dependent', 'veteran', 'faminc', 'md_faminc',
       'median_hh_inc'],
      dtype='object')

In [82]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124699 entries, 0 to 124698
Data columns (total 60 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Id                 124699 non-null  int64  
 1   OPEID              124699 non-null  object 
 2   INSTNM             124699 non-null  object 
 3   CITY               124699 non-null  object 
 4   ZIP                124699 non-null  object 
 5   sch_deg            101334 non-null  float64
 6   main               124699 non-null  object 
 7   NUMBRANCH          124699 non-null  int64  
 8   PREDDEG            124699 non-null  object 
 9   HIGHDEG            124699 non-null  object 
 10  CONTROL            124676 non-null  object 
 11  st_fips            124669 non-null  object 
 12  region             124669 non-null  object 
 13  LOCALE             7380 non-null    object 
 14  CCUGPROF           3559 non-null    object 
 15  RELAFFIL           917 non-null     object 
 16  AD

In [83]:
print(data.COMP_ORIG_YR4_RT.value_counts())
print(data.COMP_ORIG_YR4_RT.isnull().sum())

PrivacySuppressed    14586
0                     4774
1                      417
0.5                    360
0.666666666667         198
                     ...  
0.594805194805           1
0.536858974359           1
0.519362186788           1
0.022684310019           1
0.463327370304           1
Name: COMP_ORIG_YR4_RT, Length: 32741, dtype: int64
38871


In [84]:
# Convert column types, based on data dictionary
data.CCUGPROF = data.CCUGPROF.astype('category')
data.PREDDEG = data.PREDDEG.astype('category')
data.HIGHDEG = data.HIGHDEG.astype('category')
data.CONTROL = data.CONTROL.astype('category')
data.st_fips = data.st_fips.astype('category')
data.region = data.region.astype('category')
data.LOCALE = data.LOCALE.astype('category')
data.RELAFFIL = data.RELAFFIL.astype('category')
data.main = data.main.astype('bool')
#data.CURROPER = data.CURROPER.astype('bool')
#data.COMP_ORIG_YR4_RT = data.COMP_ORIG_YR4_RT.astype('float')
#data.WDRAW_ORIG_YR4_RT = data.WDRAW_ORIG_YR4_RT.astype('float')
#data.ENRL_ORIG_YR4_RT= data.ENRL_ORIG_YR4_RT.astype('float')
#data.INC_PCT_LO = data.INC_PCT_LO.astype('float')
#data.DEP_STAT_PCT_IND= data.DEP_STAT_PCT_IND.astype('float')
#data.DEP_INC_PCT_LO = data.DEP_INC_PCT_LO.astype('float')
#data.IND_INC_PCT_LO = data.IND_INC_PCT_LO.astype('float')
#data.PAR_ED_PCT_1STGEN = data.PAR_ED_PCT_1STGEN.astype('float')
#data.DEP_INC_AVG = data.DEP_INC_AVG.astype('float')
#data.IND_INC_AVG = data.IND_INC_AVG.astype('float')
#data.DEBT_MDN = data.DEBT_MDN.astype('float')
#data.GRAD_DEBT_MDN = data.GRAD_DEBT_MDN.astype('float')
#data.WDRAW_DEBT_MDN = data.WDRAW_DEBT_MDN.astype('float')
#data.loan_ever = data.loan_ever.astype('float')
#data.age_entry = data.age_entry.astype('float')
#data.female = data.female.astype('float')
#data.married = data.married.astype('float')
#data.dependent = data.dependent.astype('float')
#data.veteran = data.veteran.astype('float')
#data.faminc = data.faminc.astype('float')
#data.md_faminc = data.md_faminc.astype('float')
#data.median_hh_inc = data.median_hh_inc.astype('float')

In [85]:
col_names = ['Id','OPEID','Name','City','Zip','Predominant degree', 'Main branch', 'NumBranches',\
            'Primary degree type', 'High degree', 'Control', 'StateCode','Region','City size', \
            'Carnegie School Type', 'Religious Affiliation', 'AdmissionRate', 'ACTMidpoint', 'SATAvg',\
            'Enrollment-DegreeSeeking', 'Enrollment-Undergrad', \
            'CURROPER', 'NetPrice-Public','NetPrice-Private', 'AvgCost-AY', 'AvgCost-PY', 'TuitionRevenue/FTE',\
            'Expenditure/FTE','AvgFaculty Salary', 'Percent Fulltime Faculty', 'Completion-4yr', 'Completion<4yr','RetentionFT-4yr', \
            'RetentionFT-<4yr','RetentionPT-4yr', 'RetentionPT-<4yr', 'PercentFedLoan','PercentAge>25', \
            'PercentCompleted4yr', 'PercentWithdraw4yr', 'PercentEnrolled4yr',\
            'PercentAidedLowIncome', 'PercentIndependent', 'PercentDependentLowIncome','PercentIndepentdentLowIncome',\
            'PercentFirstGen','AvgIncomeDependent','AvgIncomeIndep','MedianDebt', \
            'MedianDebtCompleters', 'MedianDebtNotComplete', 'PercentReceiveFedLoan','AvgAgeEntry', 'PercentFemale',\
            'PercentMarried','PercentDependent','PercentVeteran', 'AvgFamilyIncome','MedianFamilyIncome',\
            'MedianHouseholdIncome']
data.columns = col_names
data.set_index('Id')

,OPEID,Name,City,Zip,Predominant degree,Main branch,NumBranches,Primary degree type,High degree,Control,...,MedianDebtNotComplete,PercentReceiveFedLoan,AvgAgeEntry,PercentFemale,PercentMarried,PercentDependent,PercentVeteran,AvgFamilyIncome,MedianFamilyIncome,MedianHouseholdIncome
Id,,,,,,,,,,,,,,,,,,,,,
1,01230800,COMMUNITY COLLEGE OF THE AIR FORCE,MONTGOMERY,361126613,NaN,True,1,Predominantly associate's-degree granting,Associate degree,Public,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00100200,ALABAMA A & M UNIVERSITY,NORMAL,35762,NaN,True,1,Predominantly bachelor's-degree granting,Graduate degree,Public,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00105200,UNIVERSITY OF ALABAMA AT BIRMINGHAM,BIRMINGHAM,352940110,NaN,True,2,Predominantly bachelor's-degree granting,Graduate degree,Public,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00574900,ALABAMA AVIATION AND TECHNICAL COLLEGE,OZARK,36360,NaN,True,1,Predominantly certificate-degree granting,Associate degree,Public,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,02503400,SOUTHERN CHRISTIAN UNIVERSITY,MONTGOMERY,361173553,NaN,True,1,Predominantly bachelor's-degree granting,Graduate degree,Private nonprofit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124695,157107,Georgia Military College-Columbus Campus,Columbus,31909,NaN,True,8,Not classified,Non-degree-granting,Public,...,5500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124696,157101,Georgia Military College-Valdosta Campus,Valdosta,31605,NaN,True,8,Not classified,Non-degree-granting,Public,...,5500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124697,157105,Georgia Military College-Warner Robins Campus,Warner Robins,31093,NaN,True,8,Not classified,Non-degree-granting,Public,...,5500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
# Convert categories to boolean types
#data.HBCU = data.HBCU.astype('bool')
#data.PBI = data.PBI.astype('bool')
#data.ANNHI = data.ANNHI.astype('bool')
#data.TRIBAL = data.TRIBAL.astype('bool')
#data.AANAPII = data.AANAPII.astype('bool')
#data.HSI = data.HSI.astype('bool')
#data.NANTI = data.NANTI.astype('bool')
#data.MENONLY = data.MENONLY.astype('bool')
#data.WOMENONLY = data.WOMENONLY.astype('bool')

In [87]:
# Create a dataframe with only closed schools
#print(data['CURROPER'])
curroper = {"Currently certified as operating": 1, "Not currently certified as an operating institution": 0}
data['CURROPER'] = data['CURROPER'].replace(curroper)
data['CURROPER'] = data['CURROPER'].astype('bool')
print(data['CURROPER'])
closed = data[~data['CURROPER']]
operational = data[data['CURROPER']]
print("Data: ", data.shape)
print("Open schools: ", operational.shape)
print("Closed schools: ", closed.shape)

0                          Currently certified as operating
1                          Currently certified as operating
2                          Currently certified as operating
3         Not currently certified as an operating instit...
4                          Currently certified as operating
                                ...                        
124694                     Currently certified as operating
124695                     Currently certified as operating
124696                     Currently certified as operating
124697                     Currently certified as operating
124698                     Currently certified as operating
Name: CURROPER, Length: 124699, dtype: object
0          True
1          True
2          True
3         False
4          True
          ...  
124694     True
124695     True
124696     True
124697     True
124698     True
Name: CURROPER, Length: 124699, dtype: bool
Data:  (124699, 60)
Open schools:  (101121, 60)
Closed schools:  (23578, 6

In [88]:
print(len(closed.OPEID.unique()))

5202


In [89]:
# Not working, try again later
closed.to_csv('data/interim/closed.filtered.csv')
operational.to_csv('data/interim/operational.filtered.csv')

In [93]:
# Counts and percents unique values
# Ranges of values

closed.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,23578.0,49811.772330,33055.783377,4.0,20497.25000,45606.5000,75408.25000,124692.0
Predominant degree,18503.0,1.789007,0.826354,1.0,1.00000,2.0000,3.00000,3.0
NumBranches,23578.0,4.849563,12.537724,1.0,1.00000,1.0000,3.00000,141.0
AdmissionRate,4080.0,0.693199,0.286237,0.0,0.49805,0.7000,0.92905,10.0
ACTMidpoint,322.0,20.773292,2.951556,9.0,19.00000,21.0000,23.00000,32.0
SATAvg,464.0,1001.066810,113.435312,560.0,937.00000,990.0000,1056.00000,1405.0
Enrollment-DegreeSeeking,20555.0,499.965604,2667.078410,0.0,55.00000,155.0000,446.50000,241832.0
Enrollment-Undergrad,1487.0,500.816409,1684.477845,0.0,49.00000,141.0000,404.50000,39386.0
NetPrice-Public,208.0,7971.524038,4197.185967,-5023.0,5552.00000,7575.5000,10377.75000,22128.0
NetPrice-Private,1906.0,17725.508919,7009.961774,-5029.0,13507.00000,18237.5000,22505.25000,44842.0


In [94]:
closed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23578 entries, 3 to 124691
Data columns (total 60 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   Id                            23578 non-null  int64   
 1   OPEID                         23578 non-null  object  
 2   Name                          23578 non-null  object  
 3   City                          23578 non-null  object  
 4   Zip                           23578 non-null  object  
 5   Predominant degree            18503 non-null  float64 
 6   Main branch                   23578 non-null  bool    
 7   NumBranches                   23578 non-null  int64   
 8   Primary degree type           23578 non-null  category
 9   High degree                   23578 non-null  category
 10  Control                       23577 non-null  category
 11  StateCode                     23577 non-null  category
 12  Region                        23577 non-null 

In [96]:
# Find the percent of null values in each column
nas=pd.DataFrame(closed.isnull().sum().sort_values(ascending=False)/len(closed),columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]

,percent
Religious Affiliation,0.999449
Carnegie School Type,0.996140
NetPrice-Public,0.991178
ACTMidpoint,0.986343
City size,0.986216
SATAvg,0.980321
RetentionPT-4yr,0.956697
AvgCost-AY,0.943888
Enrollment-Undergrad,0.936933
RetentionFT-4yr,0.925693


In [92]:
operational.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,101121.0,65273.490986,3.602785e+04,1.0,34585.000000,66740.0000,96875.00000,1.246990e+05
Predominant degree,82831.0,2.012423,8.819505e-01,1.0,1.000000,2.0000,3.00000,3.000000e+00
NumBranches,101121.0,3.478387,1.262927e+01,1.0,1.000000,1.0000,1.00000,1.410000e+02
AdmissionRate,30076.0,0.697474,2.140628e-01,0.0,0.564375,0.7223,0.85520,3.687700e+00
ACTMidpoint,15549.0,22.763329,3.259005e+00,2.0,21.000000,22.0000,24.00000,3.500000e+01
SATAvg,18134.0,1054.022223,1.269459e+02,514.0,971.000000,1036.0000,1117.00000,1.599000e+03
Enrollment-DegreeSeeking,89788.0,2529.405333,5.015666e+03,0.0,146.000000,705.0000,2556.00000,2.535940e+05
Enrollment-Undergrad,4656.0,2784.160868,4.809402e+03,0.0,144.750000,907.0000,3020.50000,4.683400e+04
NetPrice-Public,8948.0,8881.952392,4.431549e+03,-6359.0,5699.000000,8131.0000,11672.50000,3.474900e+04
NetPrice-Private,16654.0,18011.659481,7.511786e+03,-103168.0,13080.500000,18162.0000,22483.50000,8.757000e+04


In [95]:
operational.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101121 entries, 0 to 124698
Data columns (total 60 columns):
 #   Column                        Non-Null Count   Dtype   
---  ------                        --------------   -----   
 0   Id                            101121 non-null  int64   
 1   OPEID                         101121 non-null  object  
 2   Name                          101121 non-null  object  
 3   City                          101121 non-null  object  
 4   Zip                           101121 non-null  object  
 5   Predominant degree            82831 non-null   float64 
 6   Main branch                   101121 non-null  bool    
 7   NumBranches                   101121 non-null  int64   
 8   Primary degree type           101121 non-null  category
 9   High degree                   101121 non-null  category
 10  Control                       101099 non-null  category
 11  StateCode                     101092 non-null  category
 12  Region                        

In [97]:
nas=pd.DataFrame(operational.isnull().sum().sort_values(ascending=False)/len(operational),columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]

,percent
Religious Affiliation,0.991060
Carnegie School Type,0.965704
Enrollment-Undergrad,0.953956
City size,0.930232
NetPrice-Public,0.911512
AvgCost-PY,0.896945
RetentionPT-4yr,0.858031
ACTMidpoint,0.846234
NetPrice-Private,0.835306
SATAvg,0.820670


## Next steps

1. What to do with "Privacy supressed" values? How to convert to NAN?  (convert string to NAN)

2. Columns with many null values? Keep them, or get rid of them? What threshold to apply for deleting columns with 90%+ missing values